# aihc-python-sdk：快速入门


## 安装依赖

In [ ]:
! pip install python-dotenv
! pip install bce-python-sdk-next
! pip install future==1.0.0
! pip install pycryptodome==3.20.0

## 使用示例

1. 在.env文件中定义配置信息：

    ```yaml
    AK='YOUR AK'
    SK='YOUR SK'
    HOST="aihc.bj.baidubce.com"
    ```

2. 运行示例代码：

In [ ]:
# -*- coding: utf-8 -*-
import sys
import json
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

print(ak)
print(sk)

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'your resourcePoolId'
payload = {
    "queue": "default",
    "priority": "normal",
    "jobFramework": "PyTorchJob",
    "name": "test-api-llama2-7b-4",
    "jobSpec": {
        "command": "bash ./test.sh",
        "image": "registry.baidubce.com/aihc-aiak/aiak-megatron:ubuntu20.04-cu11.8-torch1.14.0-py38_v1.2.7.12_release",
        "replicas": 1,
        "resources": [
            {
                "name": "baidu.com/a800_80g_cgpu",
                "quantity": 8
            }
        ],
        "enableRDMA": true,
        "envs": [
            {
                "name": "CUDA_DEVICE_MAX_CONNECTIONS",
                "value": "1"
            }
        ]
    },
    "datasources": [
        {
            "type": "pfs",
            "name": "pfs-7xWeAt",
            "mountPath": "/mnt/cluster"
        }
    ]
}
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
logging.info('res', res)

# 获取任务列表
all_aijobs = aihc_client.get_all_aijobs(resourcePoolId='cce-e0isdmib')
print('all_aijobs: ', all_aijobs.result)
logging.info('all_aijobs: %s', all_aijobs)